- Date: 2020-05-24 00:23:26
- Author: Ben Du- Date: 2020-05-31 16:38:43
- Title: Get the Dependencies of a Python Package Without Installing It
- Slug: get-the-dependencies-of-a-Python-package-without-installing-it
- Category: Computer Science
- Tags: Computer Science, Python, package, module, dependency
- Modified: 2020-12-24 00:23:26


After installing a Python package,
you can use the command `pip3 show pkg` to check detailed information about it.
What if you want to get information about a Python package without installing?
The RESTAPI https://pypi.org/pypi/pkg/json returns detailed information of the Python package `pkg`.


Note: The recursive version of dependencies implemented bellow does not take into consideration of versions,
so it might not return the correct dependencies of a Python packages.

In [3]:
from typing import Set
import re
import requests

In [4]:
def dep(pkg: str, recursive: bool = False):
    if not recursive:
        return _dep(pkg)
    return dep_recur(pkg)


def _dep(pkg):
    url = f"https://pypi.org/pypi/{pkg}/json"
    deps = requests.get(url).json()["info"]["requires_dist"]
    if deps is None:
        return []
    return [dep for dep in deps if "extra ==" not in dep]


def _dep_recur(pkg: str, deps: Set[str]):
    for dep in _dep(pkg):
        dep = re.split(" |;|\(|\[", dep, maxsplit=1)[0]
        if dep not in deps:
            _dep_recur(dep, deps)
            deps.add(dep)


def dep_recur(pkg: str):
    deps = set()
    _dep_recur(pkg, deps)
    return deps


def _dep_recur_2(pkg: str, deps: Set[str]):
    for dep in _dep(pkg):
        dep = re.split(";| |\(|\[", dep, maxsplit=1)[0]
        if dep not in deps:
            _dep_recur_2(dep, deps)
    deps.add(pkg)


def dep_recur_2(pkg: str):
    deps = set()
    _dep_recur_2(pkg, deps)
    return deps

In [5]:
dep("pandas")

['python-dateutil (>=2.7.3)', 'pytz (>=2017.3)', 'numpy (>=1.16.5)']

In [9]:
dep("nbconvert", recursive=True)

{'MarkupSafe',
 'async-generator',
 'attrs',
 'bleach',
 'configparser',
 'defusedxml',
 'entrypoints',
 'functools32',
 'importlib-metadata',
 'ipython-genutils',
 'jinja2',
 'jsonschema',
 'jupyter-client',
 'jupyter-core',
 'jupyterlab-pygments',
 'mistune',
 'nbclient',
 'nbformat',
 'nest-asyncio',
 'packaging',
 'pandocfilters',
 'pygments',
 'pyparsing',
 'pyrsistent',
 'python-dateutil',
 'pywin32',
 'pyzmq',
 'setuptools',
 'six',
 'testpath',
 'tornado',
 'traitlets',
 'typing',
 'typing-extensions',
 'webencodings',
 'zipp'}

The difference between `dep_recur` and `dep_recur_2` is that
`dep_recur` does not include the root package into the final result 
whilel `dep_recur_2` does include the root package into the final result.

In [6]:
dep_recur("pandas")

{'numpy', 'python-dateutil', 'pytz', 'six'}

In [11]:
dep_recur_2("pandas")

{'numpy', 'pandas', 'python-dateutil', 'pytz', 'six'}

## Other Python Depedency Tools

https://github.com/naiquevin/pipdeptree

https://github.com/jazzband/pip-tools

https://stackoverflow.com/questions/21336323/show-reverse-dependencies-with-pip



## References

https://stackoverflow.com/questions/41816693/how-to-list-dependencies-for-a-python-library-without-installing